In [8]:
from google.colab import drive
drive.mount('/content/gdrive') # 點擊網址，選擇 Google 帳號登入，然後將授權碼貼回輸入框中

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
!ln -fs /content/gdrive/My\ Drive/Colab\ Notebooks/BCH\(7\,4\,3\) /acc  #執行一次就可，否則會有錯誤訊息
%cd /acc

/content/gdrive/My Drive/Colab Notebooks/BCH(7,4,3)


In [10]:
# BCH 63,45,18
# BCH_encode(Message)  產生BCH編碼 
# BCH_noise(Message)  對 Message 加入高斯雜訊
# BCH_abs(Message)   對 Message 取絕對值
# BCH_rand(Message,N)  亂數產生N個錯誤

import random
import math
import numpy as np
import random
from scipy import special

C=7
M=4
P=C-M
GX=13

class BCH:
  def toArr(_message,_C):
    # 陣列轉換
    BIN=[]
    for i in range(0,_C):
      BIN.append((_message>>i)&1)
    BIN.reverse()
    return BIN

  def encode(_message): # BCH編碼， _message 訊息
    # 初始化
    _message = _message << P
    ans = 0
    MG = 1
    # 編碼
    for i in range(C):
      if(_message & (1<<i)):
        ans = ans ^ MG
      MG = MG << 1
      if(MG >= 2**P):
        MG = MG ^ GX 
    _message = _message|ans
    return BCH.toArr(_message,C)

  def sym(_message): # 計算症狀
    ans = _message & ((1 << P) - 1)
    _GX = GX & ((1 << P) - 1)
    for i in range(P,M+P):
      if(_message & (1<<i)):
        ans = ans ^ _GX
      _GX = _GX << 1
      if(_GX & 1<<P):
        _GX = _GX ^ GX 
    return BCH.toArr(ans,P)

  def noise(_message, _r): # 加入高斯雜訊， _message 訊息 、 _r DB值
    BIN=_message
    sigma = math.sqrt((10**-(_r/10))/2)
    for i in range(len(_message)):
      if(BIN[i]==0):
        BIN[i]=1.0+float(np.random.normal(0, sigma, 1))
      else:
        BIN[i]=-1.0+float(np.random.normal(0, sigma, 1))
    return BIN

  def rand(_message, _value): # 加入雜訊， _message 訊息 、 _value 雜訊數量
    data = _message
    status = 0;
    ans=[]
    for i in range(_value):
      bit = int(random.uniform(0,C))
      ans.append(bit)
      if ((status & (1<<bit)) == 0):
        data = data ^ (1<<(bit))
        status = status ^ (1<<(bit));
    out_data = data
    ans.append(out_data)
    return ans

  def abs(_message): # 取絕對值
    BIN=_message
    for i in range(len(_message)):
      if(BIN[i]<=0):
        BIN[i]=1
      else:
        BIN[i]=0
    return BIN

  def arr_to_bin(_arr): # 陣列轉數值 
    ans=0
    for i in range(len(_arr)):
      ans = ans | (b[i] << i)
    return ans

  def Channel(_message,_w): #通道雜訊
    DB = 4
    Eb = 1
    No = math.sqrt((10**-(DB/10))/2)
    a = [0 for _ in range(0,C)]
    p = 0.5 - special.erf(math.sqrt(2.0 * Eb / No) / math.sqrt(2.0)) / 2
    BIN = []
    Y = []
    for i in range(0,C):
      BIN.append((_message >> i) & 1)
    BIN.reverse()
    while (_w > 0):
      n = int(random.uniform(0,C))
      if (a[n] == 0):
        a[n] = 1
        _w = _w - 1
    for i in range(0,C):
      U = random.random()
      F = p + (1 - p) * U
      if (a[i] == 1):
        F = p * U
      y = math.sqrt(Eb) + math.sqrt(No) * special.erfinv(2 * F - 1)
      if (BIN[i] == 1):
        Y.append(-y)
      else:
        Y.append(y)
    return Y,a
    
  def get_GX(): # 計算GX
    out=[]
    _GX = 1
    for i in range(C):
      out.append(_GX)
      _GX = _GX << 1
      if(_GX & 1<<P):
        _GX = _GX ^ GX 
    return out

In [11]:
Training = 'data/743_Training_one_to_100w.txt'
Validation = 'data/743_Validation_one_to_100w.txt'
cont = 1000000

In [12]:
y=[]
t=[]
for i in range(cont):
  a,b=BCH.Channel(0,2)
  ans=BCH.sym(BCH.arr_to_bin(a))
  ans_t=[]
  for i in range(len(ans)):
    a.append(ans[i])
  # for i in range(len(b)): #將輸出轉換為 1 or -1
    # if (b[i]==1):
    #   ans_t.append(-1)
    # else :
    #   ans_t.append(1)
  y.append(a)
  t.append(b)

In [13]:
f = open(Training,'w')
for i in range(len(y)):
  for j in range(len(y[i])):
    if (j != len(y[j])-1):
      f.write(str(y[i][j])+',')
    else :
      f.write(str(y[i][j]))
  f.write('\n')
f.close()

In [14]:
f = open(Validation,'w')
for i in range(len(t)):
  for j in range(len(t[i])):
    if (j != len(t[j])-1):
      f.write(str(t[i][j])+',')
    else :
      f.write(str(t[i][j]))
  f.write('\n')
f.close()